In [ ]:
import geopandas as gpd
from shapely.geometry import Point

def get_point(lat, lon):
    return Point(lon, lat)

def get_city_rating(lat, lon, indian_cities):
    point = get_point(lat, lon)
    for _, city in indian_cities.iterrows():
        if city['geometry'].contains(point):
            return city['Rating']
    return None

def get_country_rating(lat, lon, asia_countries):
    point = get_point(lat, lon)
    for _, country in asia_countries.iterrows():
        if country['geometry'].contains(point):
            return country['Rating']
    return None

def get_continent_rating(lat, lon, continents):
    point = get_point(lat, lon)
    for _, continent in continents.iterrows():
        if continent['geometry'].contains(point):
            return continent['Rating']
    return None

def get_combined_rating(lat, lon, indian_cities, asia_countries, continents):
    # Check city rating
    city_rating = get_city_rating(lat, lon, indian_cities)
    if city_rating:
        return city_rating
    
    # Check country rating
    country_rating = get_country_rating(lat, lon, asia_countries)
    if country_rating:
        return country_rating
    
    # Check continent rating
    return get_continent_rating(lat, lon, continents)

# Load GeoJSON or shapefiles
indian_cities = gpd.read_file('path_to_indian_cities.geojson')
indian_cities['Rating'] = [100, 90, 80, 70, 60]  # Example ratings

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
asia_countries = world[world['continent'] == 'Asia']
asia_countries['Rating'] = [75, 50, 60, 85, 40]  # Example ratings

continents = gpd.read_file('path_to_continent_geojson.geojson')
continents['Rating'] = [50, 40, 60, 70, 80]  # Example ratings

# Test with input latitude and longitude
lat, lon = 23.0225, 72.5714  # Coordinates for Ahmedabad
rating = get_combined_rating(lat, lon, indian_cities, asia_countries, continents)

print(f"Rating for the input location ({lat}, {lon}): {rating}")
